## PLX - Phuture Memecoin Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [ ]:
min_mcap =  0.25e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.9
liveness_threshold = 90 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["meme-token"]

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'base',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                -0.12,
                                100000,
                                coingecko_category)

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:
methodology_class.get_category_data()
methodology_class.get_all_coin_data()
methodology_class.filter_and_merge_coin_data(df_to_remove=[stablecoins,ctokens,atokens])
methodology_class.token_supply_check()
methodology_class.asset_maturity_check()
methodology_class.assess_liquidity()

methodology_class.category_data


Excluding aixbt, pricing data available only for 16 < 90 days
Excluding luna-by-virtuals, pricing data available only for 46 < 90 days
Excluding gout, pricing data available only for 15 < 90 days
Excluding squidgrow-2, pricing data available only for 83 < 90 days
Excluding koma-inu, pricing data available only for 45 < 90 days
Excluding banana-for-scale-2, pricing data available only for 7 < 90 days
Excluding rekt-4, pricing data available only for 13 < 90 days
Excluding moo-deng-2, pricing data available only for 71 < 90 days
Excluding vaderai-by-virtuals, pricing data available only for 14 < 90 days
Excluding stardoge, pricing data available only for 9 < 90 days
Excluding lifedog, pricing data available only for 6 < 90 days
Excluding mut, pricing data available only for 16 < 90 days
Excluding tomo-cat, pricing data available only for 5 < 90 days
Excluding super-anon, pricing data available only for 15 < 90 days
Excluding freysa-ai, pricing data available only for 4 < 90 days
solana n

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,platforms
id,,,,,,,,,
shiba-inu,SHIB,Shiba Inu,2.972000e-05,17491187481,14,17498691710,5.892560e+14,5.895088e+14,{'ethereum': '0x95ad61b0a150d79219dcf64e1e6cc0...
pepe,PEPE,Pepe,2.062000e-05,8669507789,28,8669507789,4.206900e+14,4.206900e+14,{'ethereum': '0x6982508145454ce325ddbe47a25d4e...
bonk,BONK,Bonk,4.226000e-05,3186556987,62,3911045071,7.551562e+13,9.268467e+13,{'solana': 'DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7...
floki,FLOKI,FLOKI,2.299800e-04,2223751736,75,2298496966,9.674808e+12,1.000000e+13,{'ethereum': '0xcf0c122c6b73ff809c693db761e7ba...
based-brett,BRETT,Brett,2.004210e-01,1985496773,79,1985496829,9.909996e+09,9.909997e+09,{'base': '0x532f27101965dd16442e59d40670faf5eb...
mog-coin,MOG,Mog Coin,2.370000e-06,926831669,148,926831669,3.905675e+14,3.905675e+14,{'ethereum': '0xaaee1a9723aadb7afa2810263653a3...
neiro-3,NEIRO,Neiro,1.731680e-03,728153729,168,728153729,4.206900e+11,4.206900e+11,{'ethereum': '0x812ba41e071c7b7fa4ebcfb62df5f4...
spx6900,SPX,SPX6900,6.378920e-01,594462525,198,594462525,9.309931e+08,9.309931e+08,{'ethereum': '0xe0f63a424a4439cbe457d80e4f4b51...
memecoin-2,MEME,Memecoin,1.731790e-02,564609700,204,1192338723,3.267366e+10,6.900000e+10,{'ethereum': '0xb131f4a55907b10d1f0a50d8ab8fa0...


In [6]:
methodology_class.slippage_data.head(30)

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,optimistic-ethereum,polygon-pos,optimal chain,best slippage,optimal slippage,best slippage chain
id,,,,,,,,,,,
shiba-inu,NaN,NaN,NaN,NaN,0.003730,NaN,NaN,ethereum,0.003730,0.003730,ethereum
turbo,NaN,NaN,NaN,NaN,0.000879,NaN,NaN,ethereum,0.000879,0.000879,ethereum
spx6900,NaN,NaN,-0.032067,NaN,0.000289,NaN,NaN,base,0.000289,-0.032067,ethereum
pepe,-0.065077,NaN,NaN,-0.045170,-0.000006,NaN,NaN,binance-smart-chain,-0.000006,-0.045170,ethereum
neiro-3,NaN,NaN,NaN,NaN,-0.000694,NaN,NaN,ethereum,-0.000694,-0.000694,ethereum
floki,NaN,NaN,NaN,-0.999907,-0.001741,NaN,NaN,ethereum,-0.001741,-0.001741,ethereum
dogelon-mars,NaN,NaN,NaN,-0.992203,-0.002905,NaN,-0.76469,ethereum,-0.002905,-0.002905,ethereum
based-brett,NaN,NaN,-0.004048,NaN,NaN,NaN,NaN,base,-0.004048,-0.004048,base
degen-base,NaN,NaN,-0.005934,NaN,NaN,NaN,NaN,base,-0.005934,-0.005934,base
